In [64]:
%load_ext autoreload
%autoreload 2
import sys
import torch
from torch import nn
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
from typing import Union
from pprint import pprint
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models.fc import FC
from ml_utilities.torch_models import get_model_class
from ml_utilities.output_loader.repo import Repo
from ml_utilities.output_loader.job_output import JobResult, SweepResult
from omegaconf import OmegaConf

from erank.data.datasetgenerator import DatasetGenerator

import matplotlib.pyplot as plt
gpu_id = 0
REPO = Repo(dir=Path('../../erank'), hydra_defaults=OmegaConf.load('../configs/hydra/jobname_outputdir_format.yaml'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Linear interpolation debug notebook
This notebook is used to implement linear interplation of models. 

Do linear interpolation with on MNIST. Use data from Experiment 11.7.3. 

Start from pretrained model with 100 steps.

#### Setup

In [58]:
config_yaml = """
run_config:
  exec_type: parallel # sequential
  hostname: gorilla
  gpu_ids: [0,1,2,3,4,5,6,7]
  runs_per_gpu: 3

  wandb: # wandb config for run_handler, if "wandb: null" then logging to wandb is disabled for run_handler
    init:
      tags:
        - ${config.experiment_data.experiment_tag}_exps
        - run_handler
      notes: #
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [1,2,3]

sweep:
  type: grid
  axes:
    - parameter: trainer.init_model_step
      vals: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475]
    - parameter: data.dataset_kwargs.rotation_angle
      vals: linspace(0,180,50,endpoint=True)
    - parameter: data.dataset_split.restrict_n_samples_train_task
      vals: [300] #[5, 20, 50, 100, 500, 1000, 10000, 48000]

start_num: 3 # use this to count how often this config is run
###
config:
  experiment_data:
    entity: jkuiml-fsl
    project_name: sparsity
    experiment_tag: "11.7"
    experiment_type: startnum_${start_num}
    experiment_name: mnist-${config.experiment_data.experiment_tag}.${start_num}-lenet_rottasks_ft
    experiment_dir: null
    experiment_notes: Hyperparameter search.
    job_name: null
    seed: 0
    hostname: null # the server on which the run is run, will be filled by run_handler
    gpu_id: 0

  # wandb:
  #   init:
  #     tags: # list(), used to tag wandblogger
  #       - ${config.experiment_data.experiment_tag}_exps
  #     notes: ${config.experiment_data.experiment_notes} # str, used to make notes to wandblogger
  #     group: ${config.experiment_data.experiment_tag} # null
  #     job_type: ${config.experiment_data.experiment_type} # examples: hypsearch, pretrain, eval, etc.

  #   watch:
  #     log: null #parameters #null #all
  #     log_freq: 5000

  model:
    name: fc
    model_kwargs:
      input_size: 784
      hidden_sizes:
        - 300
        - 100
      output_size: 10
      flatten_input: True
      dropout: null
      act_fn: relu

  trainer:
    training_setup: supervised
    n_steps: 2000
    log_train_step_every: 1
    log_additional_train_step_every_multiplier: 1
    log_additional_logs: True
    val_every: 5
    save_every: 5 #500
    early_stopping_patience: 200 #500
    batch_size: 128
    optimizer_scheduler:
      optimizer_name: adamw #sgd #adamw
      optimizer_kwargs:
        lr: 0.001
        weight_decay: 0.0
    
    init_model_step: XXX
    init_model: /system/user/beck/pwbeck/projects/regularization/erank/outputs/mnist-11.5.0-lenet--221015_122552/model_step_${config.trainer.init_model_step}.p

    loss: crossentropy

    metrics:
      - Accuracy
    num_workers: 4
    verbose: False

  data:
    dataset: rotatedvision
    dataset_kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      dataset: mnist
      rotation_angle: XXX
    dataset_split:
      train_val_split: 0.8
      restrict_n_samples_train_task: XXX

"""
cfg = OmegaConf.create(config_yaml)

### Load model

In [3]:
sweepr = REPO.get_output_loader(cfg)
print(sweepr)

Exp. Tag(start_num): 11.7(3)
Exp. Name: mnist-11.7.3-lenet_rottasks_ft
Training setup: supervised
Model name: fc
Dataset name: rotatedvision
Sweep type: grid
  trainer.init_model_step: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475]
  data.dataset_kwargs.rotation_angle: linspace(0,180,50,endpoint=True)
  data.dataset_split.restrict_n_samples_train_task: [300]
Num. jobs: 5400
Config updated: 2022-11-25 12:34:14
Sweep started:  2022-11-25 12:36:51



In [11]:
sw_summary = sweepr.get_summary()

100%|██████████| 5400/5400 [01:17<00:00, 69.33it/s] 


In [25]:
pv_df = sw_summary[(sw_summary['trainer.init_model_step'] == 100) & sw_summary['data.dataset_kwargs.rotation_angle'].between(0,30)]

In [26]:
pv_df.sort_values(by=['data.dataset_kwargs.rotation_angle', 'seed'])

,best_step,best_val_score,trainer.init_model_step,data.dataset_kwargs.rotation_angle,data.dataset_split.restrict_n_samples_train_task,seed
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-0-restrict_n_samples_train_task-300-seed-1--221128_013358,0,0.913250,100,0.000000,300,1
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-0-restrict_n_samples_train_task-300-seed-2--221126_064717,0,0.913250,100,0.000000,300,2
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-0-restrict_n_samples_train_task-300-seed-3--221126_203316,0,0.913250,100,0.000000,300,3
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-3.67347-restrict_n_samples_train_task-300-seed-1--221126_080453,0,0.908750,100,3.673469,300,1
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-3.67347-restrict_n_samples_train_task-300-seed-2--221126_132409,0,0.908750,100,3.673469,300,2
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-3.67347-restrict_n_samples_train_task-300-seed-3--221126_104706,0,0.908750,100,3.673469,300,3
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-7.34694-restrict_n_samples_train_task-300-seed-1--221128_022832,20,0.901667,100,7.346939,300,1
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-7.34694-restrict_n_samples_train_task-300-seed-2--221127_103518,20,0.901250,100,7.346939,300,2
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-7.34694-restrict_n_samples_train_task-300-seed-3--221126_003932,15,0.900250,100,7.346939,300,3
mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-11.0204-restrict_n_samples_train_task-300-seed-1--221127_133713,20,0.895833,100,11.020408,300,1


In [27]:
sweepr.find_jobs('init_model_step-100-rotation_angle-25')

['/system/user/publicwork/beck/projects/regularization/erank/outputs/mnist-11.7.3-lenet_rottasks_ft--221125_123651/outputs/mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-25.7143-restrict_n_samples_train_task-300-seed-2--221127_232024',
 '/system/user/publicwork/beck/projects/regularization/erank/outputs/mnist-11.7.3-lenet_rottasks_ft--221125_123651/outputs/mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-25.7143-restrict_n_samples_train_task-300-seed-3--221126_021736',
 '/system/user/publicwork/beck/projects/regularization/erank/outputs/mnist-11.7.3-lenet_rottasks_ft--221125_123651/outputs/mnist-11.7.3-lenet_rottasks_ft--init_model_step-100-rotation_angle-25.7143-restrict_n_samples_train_task-300-seed-1--221125_232309']

In [28]:
jobs_pretrainsteps100_rotangle25 = sweepr.get_jobs('init_model_step-100-rotation_angle-25')

In [31]:
# select best models trained by 3 different seeds 
# these should all be linear mode connected
model_a = jobs_pretrainsteps100_rotangle25[0].best_model
model_b = jobs_pretrainsteps100_rotangle25[1].best_model
model_c = jobs_pretrainsteps100_rotangle25[2].best_model

In [32]:
model_a

FC(
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=300, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=300, out_features=100, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [35]:
# sanity check: values should not be the same
# model_a.state_dict()['fc.1.weight'], model_b.state_dict()['fc.1.weight'] # check successful

### Load dataset

In [37]:
jobs_pretrainsteps100_rotangle25[0].override_hpparams

{'trainer.init_model_step': 100,
 'data.dataset_kwargs.rotation_angle': 25.714285714285715,
 'data.dataset_split.restrict_n_samples_train_task': 300}

In [59]:
data_cfg = cfg.config.data

In [62]:
OmegaConf.update(data_cfg, 'dataset_kwargs.rotation_angle', 25.714285714285715)
OmegaConf.update(data_cfg, 'dataset_split.restrict_n_samples_train_task', 300)

In [63]:
print(OmegaConf.to_yaml(data_cfg))

dataset: rotatedvision
dataset_kwargs:
  data_root_path: /system/user/beck/pwbeck/data
  dataset: mnist
  rotation_angle: 25.714285714285715
dataset_split:
  train_val_split: 0.8
  restrict_n_samples_train_task: 300



In [65]:
ds_generator = DatasetGenerator(**data_cfg)
ds_generator.generate_dataset()

In [67]:
len(ds_generator.train_split), len(ds_generator.val_split)

(300, 12000)

## Model interpolation

In [69]:
%autoreload 2
from erank.mode_connectivity import interpolate_linear

In [76]:
score_fn = nn.CrossEntropyLoss()

In [77]:
weights = torch.linspace(0, 1.0, 6)
weights

tensor([0.0000, 0.2000, 0.4000, 0.6000, 0.8000, 1.0000])

In [82]:
interpolate_linear(model_a, model_b, ds_generator.train_split, score_fn, weights, {'val': ds_generator.val_split})

cuda:0
Done.


In [ ]:
score_fn.device()